# How to use GMSH Python API to generate complex meshes

This tutorial is based on https://jsdokken.com/src/tutorial_gmsh.html.

In [3]:
# Modules to be imported in order
import warnings
warnings.filterwarnings("ignore")
import gmsh
gmsh.initialize()

# Additional module
import numpy as np
from dolfinx import io
from dolfinx.io import gmshio
from mpi4py import MPI

# The next step is to create the rectangular channel of the benchmark.
# In GMSH, there are two kernels for geometry computations:
#   1. The built_in kernel (gmsh.model.geo)
#   2. The OpenCascade kernel (gmsh.model.opencascade).
# In this tutorial, we will use the the second kernel, as it is better suited.
gmsh.model.add("DFG 3D")
L, B, H, r = 2.5, 0.41, 0.41, 0.05

# (0, 0, 0) are the the (x, y, z) coordinates of the first face
# The next three arguments describes the box dimensions

channel = gmsh.model.occ.addBox(0, 0, 0, L, B, H)
cylinder = gmsh.model.occ.addCylinder(0.5, 0, 0.2, 0, B, 0, r) # The last scalar defines the radius of the cylinder

# We are only interested in the fluid volume in the channel, which whould be the channel excluding the sphere. We use the GMSH command BooleanDifference for this:
fluid = gmsh.model.occ.cut([(3, channel)], [(3, cylinder)])
# The first argument [(3, channel)] is a list of tuples, where the first argument is the geometrical dimension of the entity (Point=0, Line=1, Surface=2, Volume=3). and channel is a unique integer identifying the channel.
# Similarly, the second argument is the list of tuples of entities we would like to exclude from the newly created fluid volume.

# The next step is to tag physical entities, such as the fluid volume, and inlets, outlets, channel walls and obstacle walls.
# We start by finding the volumes, which after the cut-operation is only the fluid volume. We could have kept the other volumes by supply keyword arguments to the cutoperation.
# We also need to syncronize the CAD module before tagging entities.

gmsh.model.occ.synchronize()

# Physical entity: volume
volumes = gmsh.model.getEntities(dim=3)
assert(volumes == fluid[0])
fluid_marker = 11
gmsh.model.addPhysicalGroup(volumes[0][0], [volumes[0][1]], fluid_marker)
gmsh.model.setPhysicalName(volumes[0][0], fluid_marker, "Fluid volume")

# Physical entity: surface
# For the surfaces, we start by finding all surfaces, and then compute the geometrical center such that we can indentify which are inlets, outlets, walls and the obstacle.
# As the walls will consist of multiple surfaces, and the obstacle is circular, we need to find all entites before addin the physical group.
surfaces = gmsh.model.occ.getEntities(dim=2)
inlet_marker, outlet_marker, wall_marker, obstacle_marker = 1, 3, 5, 7
walls = []
obstacles = []
for surface in surfaces:
    com = gmsh.model.occ.getCenterOfMass(surface[0], surface[1])
    if np.allclose(com, [0, B/2, H/2]):
        gmsh.model.addPhysicalGroup(surface[0], [surface[1]], inlet_marker)
        inlet = surface[1]
        gmsh.model.setPhysicalName(surface[0], inlet_marker, "Fluid inlet")
    elif np.allclose(com, [L, B/2, H/2]):
        gmsh.model.addPhysicalGroup(surface[0], [surface[1]], outlet_marker)
        gmsh.model.setPhysicalName(surface[0], outlet_marker, "Fluid outlet")
    elif np.isclose(com[2], 0) or np.isclose(com[1], B) or np.isclose(com[2], H) or np.isclose(com[1],0):
        walls.append(surface[1])
    else:
        obstacles.append(surface[1])
gmsh.model.addPhysicalGroup(2, walls, wall_marker)
gmsh.model.setPhysicalName(2, wall_marker, "Walls")
gmsh.model.addPhysicalGroup(2, obstacles, obstacle_marker)
gmsh.model.setPhysicalName(2, obstacle_marker, "Obstacle")

# The final step is to set mesh resolutions. We will use GMSH Fields to do this.
# One can alternatively set mesh resolutions at points with the command gmsh.model.occ.mesh.setSize.
# We start by specifying a distance field from the obstacle surface
distance = gmsh.model.mesh.field.add("Distance")
gmsh.model.mesh.field.setNumbers(distance, "FacesList", obstacles)

resolution = r/10
threshold = gmsh.model.mesh.field.add("Threshold")
gmsh.model.mesh.field.setNumber(threshold, "IField", distance)
gmsh.model.mesh.field.setNumber(threshold, "LcMin", resolution)
gmsh.model.mesh.field.setNumber(threshold, "LcMax", 20*resolution)
gmsh.model.mesh.field.setNumber(threshold, "DistMin", 0.5*r)
gmsh.model.mesh.field.setNumber(threshold, "DistMax", r)

inlet_dist = gmsh.model.mesh.field.add("Distance")
gmsh.model.mesh.field.setNumbers(inlet_dist, "FacesList", [inlet])
inlet_thre = gmsh.model.mesh.field.add("Threshold")
gmsh.model.mesh.field.setNumber(inlet_thre, "IField", inlet_dist)
gmsh.model.mesh.field.setNumber(inlet_thre, "LcMin", 5*resolution)
gmsh.model.mesh.field.setNumber(inlet_thre, "LcMax", 10*resolution)
gmsh.model.mesh.field.setNumber(inlet_thre, "DistMin", 0.1)
gmsh.model.mesh.field.setNumber(inlet_thre, "DistMax", 0.5)

minimum = gmsh.model.mesh.field.add("Min")
gmsh.model.mesh.field.setNumbers(minimum, "FieldsList", [threshold, inlet_thre])
gmsh.model.mesh.field.setAsBackgroundMesh(minimum)

gmsh.model.occ.synchronize()
gmsh.model.mesh.generate(3)

gmsh.write("mesh3D.msh")


domain, __, facet_tag = gmshio.read_from_msh("mesh3D.msh", MPI.COMM_WORLD, gdim = 3)
# The solution can be written to a file, and visualize it with ParaView or VisIt:
with io.XDMFFile(domain.comm, "channel_cylinder3D.xdmf", "w") as file:
    file.write_mesh(domain)
    file.write_meshtags(facet_tag)


Info    : Meshing 1D...                                                                                                      
Info    : [  0%] Meshing curve 13 (Circle)
Info    : [ 10%] Meshing curve 14 (Line)
Info    : [ 20%] Meshing curve 15 (Circle)
Info    : [ 30%] Meshing curve 16 (Line)
Info    : [ 30%] Meshing curve 17 (Line)
Info    : [ 40%] Meshing curve 18 (Line)
Info    : [ 50%] Meshing curve 19 (Line)
Info    : [ 50%] Meshing curve 20 (Line)
Info    : [ 60%] Meshing curve 21 (Line)
Info    : [ 70%] Meshing curve 22 (Line)
Info    : [ 70%] Meshing curve 23 (Line)
Info    : [ 80%] Meshing curve 24 (Line)
Info    : [ 90%] Meshing curve 25 (Line)
Info    : [ 90%] Meshing curve 26 (Line)
Info    : [100%] Meshing curve 27 (Line)
Info    : Done meshing 1D (Wall 0.0166285s, CPU 0.017366s)
Info    : Meshing 2D...
Info    : [  0%] Meshing surface 7 (Cylinder, Frontal-Delaunay)
Info    : [ 20%] Meshing surface 8 (Plane, Frontal-Delaunay)
Info    : [ 30%] Meshing surface 9 (Plane, Fron

Info    : Done writing 'mesh3D.msh'
Info    : Reading 'mesh3D.msh'...
Info    : 33 entities
Info    : 36304 nodes
Info    : 213836 elements
Info    : Done reading 'mesh3D.msh'                                                                              


TypeError: write_meshtags(): incompatible function arguments. The following argument types are supported:
    1. write_meshtags(self, meshtags: dolfinx.cpp.mesh.MeshTags_int32, x: dolfinx.cpp.mesh.Geometry_float32, geometry_xpath: str, xpath: str = '/Xdmf/Domain') -> None
    2. write_meshtags(self, meshtags: dolfinx.cpp.mesh.MeshTags_int32, x: dolfinx.cpp.mesh.Geometry_float64, geometry_xpath: str, xpath: str = '/Xdmf/Domain') -> None

Invoked with types: dolfinx.io.utils.XDMFFile, dolfinx.cpp.mesh.MeshTags_int32, dolfinx.mesh.MeshTags, str, str